In [ ]:
import tkinter as tk
from tkinter import messagebox
from fpdf import FPDF
from pymongo import MongoClient
import re

# MongoDB Setup
client = MongoClient("mongodb://localhost:27017/")
db = client["billing_app"]
collection = db["bills"]

class BillingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Billing Application")

        self.entries = {}

        # Customer Info Section
        frame = tk.Frame(root)
        frame.pack(padx=10, pady=10)

        self.add_label_entry(frame, "Organization Name")
        self.add_label_entry(frame, "Customer Name")
        self.add_label_entry(frame, "Mobile Number")
        self.add_label_entry(frame, "Mode of Payment")
        self.add_label_entry(frame, "Card Number (if Card)")

        # Item Entry Section
        self.items = []
        self.quantities = []
        self.prices = []

        item_frame = tk.LabelFrame(root, text="Items")
        item_frame.pack(padx=10, pady=10)

        self.item_name = tk.Entry(item_frame, width=20)
        self.item_name.grid(row=0, column=0)
        self.item_quantity = tk.Entry(item_frame, width=10)
        self.item_quantity.grid(row=0, column=1)
        self.item_price = tk.Entry(item_frame, width=10)
        self.item_price.grid(row=0, column=2)
        tk.Button(item_frame, text="Add Item", command=self.add_item).grid(row=0, column=3)

        self.item_listbox = tk.Listbox(item_frame, width=60)
        self.item_listbox.grid(row=1, column=0, columnspan=4, pady=5)

        # Button to Generate Receipt
        tk.Button(root, text="Generate Receipt", command=self.generate_receipt).pack(pady=10)

    def add_label_entry(self, frame, label_text):
        tk.Label(frame, text=label_text).pack()
        entry = tk.Entry(frame, width=40)
        entry.pack()
        self.entries[label_text] = entry

    def add_item(self):
        name = self.item_name.get().strip()
        try:
            quantity = int(self.item_quantity.get())
            price = float(self.item_price.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Enter valid quantity and price.")
            return

        if not name:
            messagebox.showerror("Missing Item", "Item name cannot be empty.")
            return

        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
        self.item_listbox.insert(tk.END, f"{name} x {quantity} @ Rs.{price} = Rs.{quantity * price}")
        self.item_name.delete(0, tk.END)
        self.item_quantity.delete(0, tk.END)
        self.item_price.delete(0, tk.END)

    def validate_mobile(self, mobile):
        return re.match(r'^\d{10}$', mobile)

    def generate_receipt(self):
        org_name = self.entries["Organization Name"].get()
        cust_name = self.entries["Customer Name"].get()
        mobile = self.entries["Mobile Number"].get()
        mode = self.entries["Mode of Payment"].get().lower()
        card_number = self.entries["Card Number (if Card)"].get()

        if not org_name or not cust_name or not mobile or not mode:
            messagebox.showerror("Missing Fields", "Please fill in all required fields.")
            return

        if not self.validate_mobile(mobile):
            messagebox.showerror("Invalid Mobile", "Enter a valid 10-digit mobile number.")
            return

        if mode == "card" and not re.match(r'^\d{16}$', card_number):
            messagebox.showerror("Invalid Card", "Card number must be 16 digits.")
            return

        total = sum(q * p for q, p in zip(self.quantities, self.prices))

        # Save to MongoDB
        bill_data = {
            "organization": org_name,
            "customer": cust_name,
            "mobile": mobile,
            "payment_mode": mode,
            "card_number": card_number[-4:] if mode == "card" else None,
            "items": [{"name": n, "qty": q, "price": p} for n, q, p in zip(self.items, self.quantities, self.prices)],
            "total": total
        }
        collection.insert_one(bill_data)

        # Generate PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=14)
        pdf.cell(200, 10, txt=f"{org_name} - Receipt", ln=True, align='C')
        pdf.set_font("Arial", size=10)
        pdf.cell(0, 10, txt=f"Customer Name: {cust_name}", ln=True)
        pdf.cell(0, 10, txt=f"Mobile Number: {mobile}", ln=True)
        pdf.cell(0, 10, txt=f"Payment Mode: {mode.title()}", ln=True)
        if mode == "card":
            pdf.cell(0, 10, txt=f"Card Number: XXXX-XXXX-XXXX-{card_number[-4:]}", ln=True)
        pdf.cell(0, 10, txt="Items:", ln=True)
        for name, qty, price in zip(self.items, self.quantities, self.prices):
            pdf.cell(0, 10, txt=f"{name} x {qty} @ Rs.{price} = Rs.{qty * price}", ln=True)
        pdf.cell(0, 10, txt=f"Total Amount: Rs.{total}", ln=True)

        pdf.output("receipt.pdf")
        messagebox.showinfo("Success", "Receipt generated and saved as 'receipt.pdf'.")

# Main App Execution
if __name__ == "__main__":
    root = tk.Tk()
    app = BillingApp(root)
    root.mainloop()